In [1]:
import json
import os

dirpath = os.getcwd() + '\\lemmas.json'

with open(dirpath, 'r') as f:
    documents = json.load(f)
    texts = [' '.join(doc) for doc in documents]
    
with open('swl_optimum.txt', 'r', encoding='utf-8') as f:
    swl = f.read().split('\n')

In [2]:
import pandas as pd

# Convert to a dataframe
df = pd.DataFrame(texts, columns=['headline_text'])
df.head()

,headline_text
0,март год произойти полный солнечный затмение о...
1,сентябрь состояться полный лунный затмение он ...
2,март год произойти полный солнечный затмение о...
3,июль год произойти самый продолжительный в нын...
4,первый упоминание о солнечный затмение относит...


### Create a custom embedding model

In [3]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = embedding_model.encode(df['headline_text'], show_progress_bar=True)

In [ ]:
import umap
umap_embeddings = umap.UMAP(n_neighbors=15,
                            n_components=5,
                            metric='cosine').fit_transform(embeddings)

### Create a custom vectorizer model

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words=swl, ngram_range=(1,2), min_df=3, max_df=0.9) 

### Build a BERTopic model

In [6]:
from bertopic import BERTopic

topic_model = BERTopic(language='multilingual', vectorizer_model=vectorizer_model,
                       embedding_model=None, calculate_probabilities=True,
                       verbose=True, top_n_words=15)

topics, probs = topic_model.fit_transform(df['headline_text'])

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

2022-06-18 21:54:22,788 - BERTopic - Transformed documents to Embeddings
2022-06-18 21:54:39,875 - BERTopic - Reduced dimensionality
2022-06-18 21:54:40,019 - BERTopic - Clustered reduced embeddings


### Print the results

In [7]:
# -1 refers to all outliers and should typically be ignored
freq = topic_model.get_topic_info()
freq.head(10)

,Topic,Count,Name
0,0,859,0_ген_клетка_животное_геном
1,1,108,1_звезда_галактика_карлик_планета
2,-1,82,-1_затмение_нейтрино_луна_лунный
3,2,44,2_нейтрон_нейтрино_коллайдер_кварк
4,3,42,3_нанотрубка_графен_углеродный_соединение
5,4,26,4_сверхпроводник_сверхпроводимость_сверхпровод...
6,5,20,5_капля_вода_жидкость_плёнка
7,6,19,6_клетка_белок_лауреат_калибровочный
8,7,19,7_дыра_чёрный дыра_звезда_гравитационный волна
9,8,15,8_кварк_бозон_отклонение_стандартный модель


In [10]:
topic_model.get_topics()

{-1: [('затмение', 0.04849438186316408),
  ('нейтрино', 0.02831109546162267),
  ('луна', 0.023900997099854018),
  ('лунный', 0.019704416269656574),
  ('тень', 0.017943291111351175),
  ('солнечный', 0.01693390338436965),
  ('минута', 0.015719225661529383),
  ('солнце', 0.015128863052827483),
  ('водород', 0.01375182032808399),
  ('эйнштейн', 0.01267344017031089),
  ('час', 0.012586981819066768),
  ('чандра', 0.0118051731534514),
  ('лазер', 0.01172178984195314),
  ('солнечный затмение', 0.011702549673563824),
  ('космический', 0.011004232694241216)],
 0: [('ген', 0.03533890475848215),
  ('клетка', 0.0332526522493119),
  ('животное', 0.02793192190546798),
  ('геном', 0.01633204230050336),
  ('белок', 0.015573903783745669),
  ('организм', 0.01539341233676164),
  ('эволюционный', 0.014245134504712629),
  ('растение', 0.014223386802453851),
  ('вода', 0.013053712813812209),
  ('мозг', 0.012521420535253317),
  ('признак', 0.012494936258790505),
  ('мутация', 0.012472504205906378),
  ('мышь',

In [11]:
# Select the most frequent topic
topic_model.get_topic(0)

[('ген', 0.03533890475848215),
 ('клетка', 0.0332526522493119),
 ('животное', 0.02793192190546798),
 ('геном', 0.01633204230050336),
 ('белок', 0.015573903783745669),
 ('организм', 0.01539341233676164),
 ('эволюционный', 0.014245134504712629),
 ('растение', 0.014223386802453851),
 ('вода', 0.013053712813812209),
 ('мозг', 0.012521420535253317),
 ('признак', 0.012494936258790505),
 ('мутация', 0.012472504205906378),
 ('мышь', 0.012445743671084704),
 ('древний', 0.01243268879586382),
 ('предок', 0.012261796472572242)]